In [5]:
# ------------------ SISTEMA & UTILIDADES ------------------
import warnings, os, sys
from pathlib import Path
warnings.filterwarnings("ignore")      # silencia avisos

# ------------------ MANIPULAÇÃO DE DADOS ------------------
import pandas as pd
import numpy as np

# ------------------ VISUALIZAÇÃO --------------------------
import matplotlib.pyplot as plt
import seaborn as sns

# ------------------ MACHINE LEARNING ----------------------
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from catboost import CatBoostRegressor, Pool

# ------------------ OUTROS -------------------------------
import scipy.stats as stats
import joblib                    # salvar / carregar modelos

print("✔️ Todas as bibliotecas importadas!")


✔️ Todas as bibliotecas importadas!


In [6]:
# ---------- caminho para o CSV ----------
caminho = Path(r"C:\wamp64\www\previsor-enem\dados\MICRODADOS_ENEM_2023.csv")

# ---------- variáveis dependentes ----------
alvos = [
    "NU_NOTA_CN","NU_NOTA_CH","NU_NOTA_LC","NU_NOTA_MT",
    "NU_NOTA_COMP1","NU_NOTA_COMP2","NU_NOTA_COMP3",
    "NU_NOTA_COMP4","NU_NOTA_COMP5","NU_NOTA_REDACAO"
]

# ---------- variáveis independentes ----------
cat_cols = [
    "TP_FAIXA_ETARIA","TP_SEXO","TP_ESTADO_CIVIL","TP_COR_RACA",
    "TP_NACIONALIDADE","TP_ST_CONCLUSAO","TP_ANO_CONCLUIU",
    "TP_ESCOLA","TP_ENSINO","IN_TREINEIRO","CO_MUNICIPIO_ESC",
    "TP_LOCALIZACAO_ESC","TP_SIT_FUNC_ESC",
    "TP_PRESENCA_CN","TP_PRESENCA_CH","TP_PRESENCA_LC","TP_PRESENCA_MT",
    "TP_STATUS_REDACAO",
    "Q001","Q002","Q003","Q004","Q005","Q006","Q007","Q008",
    "Q009","Q010","Q011","Q012","Q013","Q014","Q015","Q016",
    "Q017","Q018","Q019","Q020","Q021","Q022","Q023","Q024","Q025",
]

cols_modelo = cat_cols + alvos
dtypes = {c: "category" for c in cat_cols}


In [7]:
# ---------- leitura ----------
df = pd.read_csv(
    caminho,
    sep=";",
    encoding="latin-1",
    usecols=cols_modelo,
    dtype=dtypes,         # 43 colunas como 'category'
    engine="pyarrow"      # mais rápido; remova se não tiver pandas >= 2.2
)

print(f"✔️ CSV carregado!\nShape: {df.shape}")
print(f"Memória usada: {df.memory_usage(deep=True).sum()/1e6:.1f} MB")

# ---------- INFO GERAL ----------
print("\n── INFO GERAL ──")
df.info(max_cols=0)

# ---------- CARDINALIDADE TOP‑10 ----------
print("\n── CARDINALIDADE (top‑10) ──")
top_card = (
    df[cat_cols]
      .nunique(dropna=False)
      .sort_values(ascending=False)
      .head(10)
)
print(top_card.to_string())

# ---------- DESCRITIVO DAS NOTAS ----------
print("\n── DESCRITIVO DAS NOTAS ──")
print(df[alvos].describe().T.round(2))


✔️ CSV carregado!
Shape: (3933955, 53)
Memória usada: 488.0 MB

── INFO GERAL ──
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3933955 entries, 0 to 3933954
Columns: 53 entries, TP_FAIXA_ETARIA to NU_NOTA_REDACAO
dtypes: category(43), float64(10)
memory usage: 465.4 MB

── CARDINALIDADE (top‑10) ──
CO_MUNICIPIO_ESC     5529
TP_FAIXA_ETARIA        20
Q005                   20
TP_ANO_CONCLUIU        18
Q006                   17
TP_STATUS_REDACAO       9
Q002                    8
Q001                    8
Q003                    6
Q004                    6

── DESCRITIVO DAS NOTAS ──
                     count    mean     std  min    25%    50%    75%     max
NU_NOTA_CN       2692427.0  495.75   87.93  0.0  440.5  493.9  551.2   868.4
NU_NOTA_CH       2822643.0  523.35   88.57  0.0  467.8  530.4  584.9   823.0
NU_NOTA_LC       2822643.0  518.15   75.45  0.0  471.4  523.1  570.3   820.8
NU_NOTA_MT       2692427.0  533.84  131.65  0.0  431.2  523.6  630.1   958.6
NU_NOTA_COMP1    282264

In [13]:
# ---------- amostragem ----------
sample_frac = 0.03
sample = df.sample(frac=sample_frac, random_state=42)
print(f"\nAmostra criada: {sample.shape[0]:,} linhas "
      f"({sample_frac:.0%} do total)")

# ---------- preencher NaNs nos categóricos ----------
for col in cat_cols:
    if "Ausente" not in sample[col].cat.categories:
        sample[col] = sample[col].cat.add_categories("Ausente")
    sample[col] = sample[col].fillna("Ausente")

na_pct = (sample[cat_cols] == "Ausente").mean().sort_values(ascending=False).head(5)
print("\nTop‑5 colunas com mais 'Ausente':")
print((na_pct*100).round(2).astype(str) + " %")

# ------------------------------------------------------------------
# 3b) REMOVER LINHAS SEM NOTA (NU_NOTA_MT)
# ------------------------------------------------------------------
antes = sample.shape[0]
sample = sample.dropna(subset=["NU_NOTA_MT"])
depois = sample.shape[0]

print(f"Linhas removidas por falta da nota de Matemática: {antes-depois:,}")
print(f"Sample final para modelagem: {depois:,} linhas")




Amostra criada: 118,019 linhas (3% do total)

Top‑5 colunas com mais 'Ausente':
TP_SIT_FUNC_ESC        75.8 %
CO_MUNICIPIO_ESC       75.8 %
TP_LOCALIZACAO_ESC     75.8 %
TP_ENSINO             66.29 %
TP_STATUS_REDACAO     28.29 %
dtype: object
Linhas removidas por falta da nota de Matemática: 37,321
Sample final para modelagem: 80,698 linhas


In [14]:
# 4) SPLIT + POOLS CATBOOST  (revisto)

# ❶ garante que todas as categorias sejam strings
for col in cat_cols:
    sample[col] = sample[col].astype(str)

# ❷ divide treino e teste
train, test = train_test_split(
    sample,
    test_size=0.20,
    random_state=42,
    stratify=sample["TP_FAIXA_ETARIA"]
)

# ❸ separa features e alvo
X_train, X_test = train.drop(alvos, axis=1), test.drop(alvos, axis=1)
y_train, y_test = train["NU_NOTA_MT"], test["NU_NOTA_MT"]

# ❹ cria Pools
pool_tr = Pool(X_train, y_train, cat_features=cat_cols)
pool_te = Pool(X_test,  y_test,  cat_features=cat_cols)

print(f"Treino: {X_train.shape}  |  Teste: {X_test.shape}")
print("Targets sem NaN?  ->", y_train.isna().sum() + y_test.isna().sum() == 0)




Treino: (64558, 43)  |  Teste: (16140, 43)
Targets sem NaN?  -> True


In [15]:
model = CatBoostRegressor(
    iterations=300,
    depth=6,
    learning_rate=0.10,
    loss_function="MAE",
    random_state=42,
    verbose=100,
    early_stopping_rounds=30
)

model.fit(pool_tr, eval_set=pool_te)

pred  = model.predict(pool_te)
mae   = mean_absolute_error(y_test, pred)
rmse  = mean_squared_error(y_test, pred, squared=False)

print(f"\nBaseline NU_NOTA_MT →  MAE: {mae:.2f}  |  RMSE: {rmse:.2f}")

print("\nTop‑15 variáveis mais importantes:")
print(model.get_feature_importance(prettified=True).head(15).to_string(index=False))



0:	learn: 104.0239814	test: 104.7193236	best: 104.7193236 (0)	total: 432ms	remaining: 2m 9s
100:	learn: 82.5406131	test: 83.5760686	best: 83.5760686 (100)	total: 23.3s	remaining: 46s
200:	learn: 81.5993374	test: 83.3122074	best: 83.3122074 (200)	total: 46.6s	remaining: 23s
299:	learn: 81.1147785	test: 83.2285514	best: 83.2270983 (292)	total: 1m 13s	remaining: 0us

bestTest = 83.22709833
bestIteration = 292

Shrink model to first 293 iterations.

Baseline NU_NOTA_MT →  MAE: 83.23  |  RMSE: 107.49

Top‑15 variáveis mais importantes:
       Feature Id  Importances
             Q006    14.218337
             Q024     9.859420
          TP_SEXO     7.255477
  TP_ANO_CONCLUIU     7.145642
TP_STATUS_REDACAO     5.891033
             Q003     5.450109
             Q002     4.893041
      TP_COR_RACA     4.787785
        TP_ESCOLA     4.668568
             Q001     4.252155
  TP_FAIXA_ETARIA     3.965471
             Q013     3.000707
             Q005     2.507616
             Q022     2.44430

Valores ausentes preenchidos com a categoria 'Ausente'.

── % 'Ausente' nas 10 colunas com mais faltantes ──
TP_SIT_FUNC_ESC        75.8 %
CO_MUNICIPIO_ESC       75.8 %
TP_LOCALIZACAO_ESC     75.8 %
TP_ENSINO             66.29 %
TP_STATUS_REDACAO     28.29 %
Q016                    0.0 %
Q009                    0.0 %
Q010                    0.0 %
Q011                    0.0 %
Q012                    0.0 %
dtype: object


In [16]:
# -------------------------------------------------------------
# 0) Pré‑requisitos — modelo 'model', conjunto base 'sample'
# -------------------------------------------------------------
df_cf = sample.copy()                 # clone p/ contrafactual

# 1) ORDEM das categorias (garante que 'A' < 'B' < ... < 'Q')
ordem_renda = list("ABCDEFGHIJKLMNOPQ")  # 17 códigos
df_cf["Q006"] = pd.Categorical(df_cf["Q006"], ordem_renda, ordered=True)

# 2) SUBIR 1 DEGRAU para quem não está no topo
def sobe_renda(x):
    idx = ordem_renda.index(x)
    return ordem_renda[min(idx + 1, len(ordem_renda) - 1)]

df_cf["Q006_cf"] = df_cf["Q006"].apply(sobe_renda).astype(str)   # nova coluna
df_cf = df_cf.drop(columns="Q006").rename(columns={"Q006_cf": "Q006"})

# 3) PREDIÇÕES
pred_base = model.predict(Pool(sample, cat_features=cat_cols))
pred_cf   = model.predict(Pool(df_cf,  cat_features=cat_cols))

ganho_individual = pred_cf - pred_base
ganho_medio = ganho_individual.mean()
print(f"Ganho médio ao subir 1 faixa de renda: +{ganho_medio:.1f} pontos em MT")

# 4) DETALHE por faixa de origem (opcional)
import pandas as pd
df_result = (
    pd.DataFrame({"faixa_original": sample["Q006"], "ganho": ganho_individual})
      .groupby("faixa_original")["ganho"]
      .agg(["count","mean","median"])
      .round(1)
      .sort_index()
)
print("\nGanho por faixa de renda original:")
print(df_result.head(10))


Ganho médio ao subir 1 faixa de renda: +10.1 pontos em MT

Ganho por faixa de renda original:
                count  mean  median
faixa_original                     
A                4687   6.6     7.0
B               23463  19.1    19.3
C               12427   7.1     7.2
D                9033   8.0     7.7
E                6333  11.6    11.3
F                3948   6.5     6.0
G                5999   0.8     0.2
H                3288  15.0    14.4
I                2140   1.7     0.8
J                1819   8.4     9.0
